In [1]:
!pip install python-dotenv
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
import warnings
warnings.filterwarnings('ignore')

# 設定 APIKEY
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

HF_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# **Intorduction to 🤗 Hugging Face and Transformers Library**

[🤗 hugging face NLP course](https://huggingface.co/learn/nlp-course/chapter1/1)


NLP 是語言學和機器學習領域，專注於理解與人類語言相關的一切。 NLP 任務的目標不僅是單獨理解單字，而且能夠理解這些單字的上下文。</p>
NLP 常見的任務有：
   1. **Classifying whole sentences 文本分類（分類整句）**：將整個句子進行分類
      - 情感分析：「這部電影很棒！」 -> positive
      - 垃圾郵件檢測：「你的 iphone 已被嚴重損壞」-> spam
   2. **Classifying each word in a sentence 單詞分類**：對一句話中的所有字進行分類
      - 語法分析：「他跑得快」-> 他（代詞）跑（動詞）得（副詞）快（形容詞）
      - 命名實體 NER
   3. **Sentence Generation**
      1. **填充遮蔽詞**：「像這種要求，我這輩子[mask]！」-> [mask] 預測為 沒聽過
      2. **自動生成**：「今天天氣如何」-> 「今天天氣非常晴朗適合外出」
      3. **翻譯**：「你好」-> 「Hello」
      4. **摘要（問答）**
         1. 從文本中提取答案 Extractive QA：「法國的首都是巴黎。   首都在哪」-> 巴黎（藉由巴黎在原文的 index 抓出來）
         2. 以生成模型進行摘要 Generative QA：「法國的首都是巴黎。   首都在哪」-> 首都在巴黎（使用生成模型，例如 chatgpt）

## **`pipeline` in Hugging Face transformers**
``transformers`` 為 🤗Hugging face 提供的套件，讓開發者可以創建、使用 Hugging face hub 上 NLP、LLM 的模型</p>
> Hugging face hub 上的模型不只有 transformer，任何人都可以上傳任何類型的模型或資料集

在 `transformers` 中最高階的函數是 `pipeline`，
該函數將使用模型需要的預處理、推理與後處理串連起來，</p>
傳入指定的 task，`pipeline` 會自動以適合的模型進行推理（預測）。

可以從 [hub](https://huggingface.co/models) 透過 Tasks、Languages 篩選找到自己想要應用的模型</p>
從 1. [task summary](https://huggingface.co/docs/transformers/task_summary) 2. [Tasks](https://huggingface.co/tasks) 找到支援的 NLP 相關任務

### **使用 pipeline 完成常見 NLP 任務**

1. 情感分析 aka 分類問題

In [3]:
from pprint import pprint
from transformers import pipeline
pipe = pipeline(task="sentiment-analysis")
pipe("this is awesome!!!")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9998723268508911}]

2. 命名實體</p>



在 ner 任務中，模型會對所有字詞（token） 進行分類，得到：</p>
1. 該字詞（token）對應的 entity
2. score 機率值
3. 以及對應到文本的起始結束位置

In [4]:
ner_pipe = pipeline(task="ner",
                # model='dslim/bert-base-NER'
                )
ner_pipe("Hugging Face is a French company based in New York City.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

[{'entity': 'I-ORG',
  'score': 0.9967675,
  'index': 1,
  'word': 'Hu',
  'start': 0,
  'end': 2},
 {'entity': 'I-ORG',
  'score': 0.92930275,
  'index': 2,
  'word': '##gging',
  'start': 2,
  'end': 7},
 {'entity': 'I-ORG',
  'score': 0.9763208,
  'index': 3,
  'word': 'Face',
  'start': 8,
  'end': 12},
 {'entity': 'I-MISC',
  'score': 0.99828726,
  'index': 6,
  'word': 'French',
  'start': 18,
  'end': 24},
 {'entity': 'I-LOC',
  'score': 0.99896204,
  'index': 10,
  'word': 'New',
  'start': 42,
  'end': 45},
 {'entity': 'I-LOC',
  'score': 0.9986792,
  'index': 11,
  'word': 'York',
  'start': 46,
  'end': 50},
 {'entity': 'I-LOC',
  'score': 0.9992418,
  'index': 12,
  'word': 'City',
  'start': 51,
  'end': 55}]

### **練習 #1**

使用 Extractive QA model 以 `pipeline` 做 question-answering 任務：
- **給定文本**：the name of repo is bert-base-uncased
- **問題目標**：問模型 repo 的名稱
- **預期答案**：bert-base-uncased

In [5]:
# TODO
# practice 1 不需要特別指定模型，pipeline 預設載入 distilbert-base-cased-distilled-squad,
# 其為 Extractive QA 類摘要模型

from transformers import pipeline

### **利用 Conversation class 與 text-generation model 實作 chatbot**

In [6]:
from pprint import pprint

from torch import cuda, bfloat16
from transformers import pipeline
from transformers import BitsAndBytesConfig, AutoConfig, AutoModelForCausalLM, AutoTokenizer
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)

cuda:0


因為載入模型較大，使用 T4 GPU 時建議進行量化，以下程式為量化處理過程，</p>
在此先不贅述，有興趣的可以參考 Hugging face 官方文件～</p>

與前面範例不同的是，模型載入方法，我們透過 `AuToModelForCausalLM` 實例化模型，將其作為參數傳入 `pipeline`。

In [7]:
model_id = 'MediaTek-Research/Breeze-7B-32k-Instruct-v1_0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model_config = AutoConfig.from_pretrained(
    model_id
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

hf_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/512M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [8]:
chatbot = pipeline(
    "text-generation",
    model=hf_model,
    tokenizer=tokenizer, # Tokenizer，要與模型匹配，主要提供 chat 模式時的特殊符號
    max_new_tokens=1024, # 模型最多可以生成多少字
    return_full_text=False # 控制 pipeline 只輸出 AI Message
)

聊天式模型，例如 ChatGPT 其實基本上是透過 text-generation 作為基礎模型，進一步訓練模型能過聊天。
所以模型的選擇，我們可以在 Huggingface 上找到 text-generation 任務的模型，應該都可以支援。</p>

比較特別的是，要做聊天任務時，模型需要一些特殊符號來區別每一段訊息是來自於 User 或是 AI 還是 System Prompt</p>
而各個模型的特殊符號不盡相同，需要去查閱官方文件。例如 Demo 使用的聯發科 Breeze 模型是透過 `[INST]` 、 `[/INST]` 以及 `<s>` 作為區隔。
所以我們在使用模型時，就會需要將文字加上這些特殊符號才能夠發揮模型聊天的能力。</p>

通常我們會使用 list of dict 的方式處存聊天的記錄，使用 role 區別 user 與 ai，content 代表內容，而 Hugging face 的模型也支援這樣的格式，例如：


```
[
    {"role": "user", "content": "嗨你好嗎"},
    {"role": "assistant", "content": "嗨您好，我是您的 AI 助理，很高興為您服務。"},
    {"role": "user", "content": "掰掰"},
    {"role": "assistant", "content": "掰掰，期待再相見"},

]

```

我們可以透過實例化 Conversation 這個 class，透過 `add_user_input` 與 `append_response` 新增歷史使用者輸入與模型回覆，將資料變為上述的資料格式再送給模型進行推理。


In [9]:
from transformers import Conversation
conversation = Conversation() # 建立一個對話 Conversation 物件

利用 `add_user_input` 新增 user 聊天記錄

In [10]:
conversation.add_user_input("provided information: the name of repo is bert-base-uncased. Based on the provided information, what is the name of repo?")
print(f"目前聊天記錄：{conversation.messages}") # conversation.messages 可以直接丟給 chatbot 得到回覆

目前聊天記錄：[{'role': 'user', 'content': 'provided information: the name of repo is bert-base-uncased. Based on the provided information, what is the name of repo?'}]


In [11]:
# 將 conversation.messages 丟給 chatbot
chatbot_result = chatbot(conversation.messages)
print(chatbot_result)

[{'generated_text': '根據提供的信息，repo的名称是"bert-base-uncased"。'}]


將 chatbot 的回覆以 `append_respons` 的方法加入 conversation 中

In [12]:
conversation.append_response(chatbot_result[0]['generated_text'])
print(f"目前聊天記錄：{conversation.messages}")

目前聊天記錄：[{'role': 'user', 'content': 'provided information: the name of repo is bert-base-uncased. Based on the provided information, what is the name of repo?'}, {'role': 'assistant', 'content': '根據提供的信息，repo的名称是"bert-base-uncased"。'}]


In [13]:
conversation.add_user_input("那什麼是 bert?")

print(f"目前聊天記錄：{conversation.messages}")

目前聊天記錄：[{'role': 'user', 'content': 'provided information: the name of repo is bert-base-uncased. Based on the provided information, what is the name of repo?'}, {'role': 'assistant', 'content': '根據提供的信息，repo的名称是"bert-base-uncased"。'}, {'role': 'user', 'content': '那什麼是 bert?'}]


In [14]:
chatbot_result = chatbot(conversation.messages)
print(f"LLM 回覆：{chatbot_result}")

print("-"*10)
conversation.append_response(chatbot_result[0]['generated_text'])
print(f"目前聊天記錄：{conversation.messages}")

LLM 回覆：[{'generated_text': ' Bert是Bidirectional Embedding Representations from Transformers的缩写，是Google自然语言处理团队所研发的一个预训练模型。它基于Transformers架构，通过同时考虑文本的前向和后向信息，实现了有针对性地语义表达和知识表达。Bert模型在各种自然语言处理任务，如情感分析、命名实体识别、问答系统等任务上表现突出，已成为当今预训练模型中的一个标准基线。'}]
----------
目前聊天記錄：[{'role': 'user', 'content': 'provided information: the name of repo is bert-base-uncased. Based on the provided information, what is the name of repo?'}, {'role': 'assistant', 'content': '根據提供的信息，repo的名称是"bert-base-uncased"。'}, {'role': 'user', 'content': '那什麼是 bert?'}, {'role': 'assistant', 'content': ' Bert是Bidirectional Embedding Representations from Transformers的缩写，是Google自然语言处理团队所研发的一个预训练模型。它基于Transformers架构，通过同时考虑文本的前向和后向信息，实现了有针对性地语义表达和知识表达。Bert模型在各种自然语言处理任务，如情感分析、命名实体识别、问答系统等任务上表现突出，已成为当今预训练模型中的一个标准基线。'}]


### **embedding model (feature extraction)**
[參考](https://huggingface.co/tasks/feature-extraction)


Embedding 是將文字轉換成向量的技術，使得文字可以在數學空間中表示。</p>這些向量捕捉了文字之間的語義關係，使得相似的文字在向量空間中更接近。常見的嵌入模型包括 Word2Vec、GloVe 和 BERT 等。

在 Retrieval-Augmented Generation (RAG) 中，我們會用 Embedding model 用來將查詢（query）和候選文檔（document）轉換成向量。</p>
通過計算這些向量的相似度，可以找出與查詢最相關的文檔。這些相關文檔隨後用來生成回答，增強生成模型的準確性和上下文相關性。

我們使用 sentence transformers 這個套件，可以從 [官方文檔](https://sbert.net/docs/pretrained_models.html) 尋找自己希望使用的 model，也可以在 hugging face 平台上搜尋支援 feature-extraction 的模型。</p>

另外 hugging face 也提供 [embedding model](https://huggingface.co/spaces/mteb/leaderboard) 的排行榜給大家參考

In [15]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [16]:
# 利用 encode 得到 sentence 的 embedding
embedding_model.encode("哈囉，這是一個句子")

array([ 0.03142083, -0.01894771, -0.00766944, ..., -0.02039895,
       -0.01210877,  0.03742645], dtype=float32)

In [17]:
query = "為什麼 ML 需要做正規化"

source_sentence = [
    'Regularization is important!',
    'Dropout is important!',
    'Missing Data Handling is important!'
]

當我們有每個句子的 embedding 後就可以透過 cosine similarity 計算每個文本的相似度。

In [18]:
import numpy as np
def calculate_cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [19]:
most_related_sentence = None
max_similarity = 0

for sentence in source_sentence:
    sim = calculate_cosine_similarity(
    embedding_model.encode(query),
    embedding_model.encode(sentence)
    )

    if sim > max_similarity:
        most_related_sentence = sentence
        max_similarity = sim

    print(f"{query} vs {sentence} similarity: {sim}")

print("="*10)
print(f"與「{query}」最相似文本：{most_related_sentence}")

為什麼 ML 需要做正規化 vs Regularization is important! similarity: 0.8665975332260132
為什麼 ML 需要做正規化 vs Dropout is important! similarity: 0.805509626865387
為什麼 ML 需要做正規化 vs Missing Data Handling is important! similarity: 0.814452588558197
與「為什麼 ML 需要做正規化」最相似文本：Regularization is important!


**作業 demo**

利用 Hugging Face 的 text-generation model 與 Sentence Transformers embedding model 實作 QA 檢索聊天機器人。</p>
基於提供的資料集，使用 Embedding Cosine Similarity 檢索參考資料，再透過 LLM 生成答案。</p>

1. Baseline
   - 將 demo 中的資料，替換成我們提供 or 自己的資料集
   - 能夠檢索相似資料
   - 基於檢索的資料進行回答
2. Advanced（Optional）
   - Embedding 怎麼儲存？每次都要重新計算嗎？
   - 該如何處理太久以前的歷史資料？
   - 利用 Gradio or Hugging Face Spaces 部署、分享 Chatbot

In [21]:
qa_data = [
    """
    The Big Bang Theory is an American television sitcom created by Chuck Lorre and Bill Prady, both of whom served as executive producers and head writers on the series, along with Steven Molaro. It aired on CBS from September 24, 2007, to May 16, 2019, running for 12 seasons and 279 episodes.[3]

The show originally centered on five characters living in Pasadena, California: Leonard Hofstadter (Johnny Galecki) and Sheldon Cooper (Jim Parsons), both physicists at Caltech, who share an apartment; Penny (Kaley Cuoco), a waitress and aspiring actress who lives across the hall; and Leonard and Sheldon's similarly geeky and socially awkward friends and coworkers, aerospace engineer Howard Wolowitz (Simon Helberg) and astrophysicist Raj Koothrappali (Kunal Nayyar).[4][5] Over time, supporting characters were promoted to starring roles, including neuroscientist Amy Farrah Fowler (Mayim Bialik), microbiologist Bernadette Rostenkowski (Melissa Rauch), and comic book store owner Stuart Bloom (Kevin Sussman).

The show was filmed in front of a live audience and produced by Chuck Lorre Productions and Warner Bros. Television. It received mixed reviews throughout its first season, but reception was more favorable in the second and third seasons. Despite early mixed reviews, seven seasons were ranked within the top ten of the final season ratings, and it ultimately reached the no. 1 spot in its eleventh season. It was nominated for the Emmy Award for Outstanding Comedy Series from 2011 to 2014 and won the Emmy Award for Outstanding Lead Actor in a Comedy Series four times for Parsons, totaling seven Emmy Awards from 46 nominations. Parsons also won the Golden Globe for Best Actor in a Television Comedy Series in 2011. A prequel series, titled Young Sheldon and based on Parsons' character Sheldon Cooper, aired from 2017 to 2024, with Parsons reprising his role as the narrating adult Sheldon. The third series in the franchise, a sequel series to Young Sheldon titled Georgie & Mandy's First Marriage, is scheduled to premiere in fall 2024, and will follow Sheldon's older brother Georgie and his wife Mandy.
    """,

    """
    Cast and characters
    Johnny Galecki as Leonard Hofstadter:[6] An experimental physicist with an IQ of 173, who received his Ph.D. when he was 24 years old. Leonard is a nerd who loves video games, comic books, and Dungeons & Dragons. Leonard is the straight man of the series, sharing an apartment in Pasadena, CA, with Sheldon Cooper. Leonard is smitten with his new neighbor Penny when they first meet, and they eventually marry.
Jim Parsons as Sheldon Cooper:[7] Originally from Galveston, Texas, Sheldon was a child prodigy with an eidetic memory who began college at the age of eleven and earned a Ph.D. at age sixteen. He is a theoretical physicist researching quantum mechanics and string theory, and, despite his IQ of 187, he finds many routine aspects of social situations difficult to grasp. He is determined to have his own way, continually boasts of his intelligence, and has an extremely ritualized way of living. Despite these quirks, he begins a relationship with Amy Farrah Fowler, and they eventually marry.
Kaley Cuoco as Penny:[8] An aspiring actress from Omaha, Nebraska. Penny moves in across the hall from Sheldon and Leonard. She waits tables and occasionally tends the bar at The Cheesecake Factory. After giving up hope of becoming a successful actress, Penny becomes a pharmaceutical sales representative. Penny becomes friends with Bernadette and Amy, and they often hang out in each other's apartments. Penny and Leonard form a relationship and eventually marry.
Simon Helberg as Howard Wolowitz:[9] An aerospace engineer who got his master's degree at the Massachusetts Institute of Technology. Howard is Jewish and lived with his mother, Debbie (Carol Ann Susi). Unlike Sheldon, Leonard, Raj, Bernadette, and Amy, Howard does not hold a doctorate. He trains as an astronaut and goes into space as a payload specialist on the International Space Station. Howard initially fancies himself as a ladies man, but he later starts dating Bernadette, and they get engaged and married. Howard also has a tendency to waste money on toys and argues with Bernadette because of his oddly low income as an engineer and her high income as a pharmaceutical biochemist.
Kunal Nayyar as Rajesh Koothrappali:[10] A particle astrophysicist originally from New Delhi, India. Initially, Raj had selective mutism, rendering him unable to talk to or be around women unless under the influence of alcohol. Raj also has very feminine tastes and often takes on a stereotypical female role in his friendship with Howard as well as in the group of four men. Raj later dates Lucy (Kate Micucci), who also suffers from social anxiety, but it eventually ends. He later speaks to Penny without alcohol, overcoming his selective mutism. He begins dating Emily Sweeney, and their relationship later becomes exclusive. In the series' final season, Raj has an on-again, off-again engagement with a fellow Indian, a hotel concierge named Anu (Rati Gupta). He also has a Yorkshire Terrier named Cinnamon.
Sara Gilbert as Leslie Winkle (recurring season 1, starring season 2, guest seasons 3, 9):[11][12][13] A physicist who works in the same lab as Leonard. In appearance, she is essentially Leonard's female counterpart and has conflicting scientific theories with Sheldon. Leslie has casual sex with Leonard and later Howard. Gilbert was promoted to a main cast member during the second season but resumed guest star status because producers could not come up with enough material for the character.[11] Gilbert returned to The Big Bang Theory for its 200th episode.[14]
Melissa Rauch as Bernadette Rostenkowski-Wolowitz (recurring season 3, starring seasons 4–12):[15] A young woman who initially is a co-worker at The Cheesecake Factory with Penny to pay her way through graduate school, where she is studying microbiology. Bernadette is introduced to Howard by Penny; at first, they do not get along, apparently having nothing in common. They date and later get engaged and married. Although generally a sweet and good-natured person, Bernadette has a short fuse and can be vindictive and lash out when provoked.
Mayim Bialik as Amy Farrah Fowler (guest star season 3, starring seasons 4–12):[16] A woman selected by an online dating site as Sheldon's perfect mate,[17] Amy is from Glendale, California. While she and Sheldon initially share social cluelessness, after befriending Penny and Bernadette, she eventually becomes more interested in social and romantic interaction. Her relationship with Sheldon slowly progresses to the point at which Sheldon considers her his girlfriend, and eventually, they get married. Amy believes she and Penny are best friends, a sentiment that Penny does not initially share. Amy has a Ph.D. in neurobiology.
Kevin Sussman as Stuart Bloom (recurring seasons 2–5, 7, starring seasons 6, 8–12):[18] A mild-mannered, under-confident owner of a comic book store. A competent artist, Stuart is a graduate of the prestigious Rhode Island School of Design. Though he is socially awkward, he possesses slightly better social skills. Stuart implies he is in financial trouble and that the comic book store now also is his home. He is later invited to join the guys' group while Howard is in space. Stuart gets a new job caring for Howard's mother later. After Mrs. Wolowitz's death, Stuart continues to live in her home, along with Howard and Bernadette, until he finds a place of his own.
Laura Spencer as Emily Sweeney (recurring seasons 7–8, 10, starring season 9):[19] A dermatologist at Huntington Hospital. Emily went to Harvard and delights in the macabre, and she states that she likes her job because she can cut things with knives. Prior to meeting Raj, Emily was set up on a blind date with Howard. After finding Emily's online dating profile, Raj has Amy contact her as his wingman instead. Their relationship becomes exclusive, but Raj later breaks up with Emily when he becomes infatuated with Claire (Alessandra Torresani), a bartender and children's author.
    """,
    """
    《宅男行不行》於美國東部時間周一晚間9時30分播放，緊跟另一部由洛爾製作的影集《好漢兩個半》[8]。2009年9月21日第三季第一集播映時，創下了CBS夜間18-49歲成人觀眾收視率的新紀錄（4.6/10），當時共有1283萬觀眾收看該劇[9]。2010年5月19日，CBS宣布會在2010～2011年檔期內，改在東部時間周四晚八點的黃金檔播放該劇。2011年1月12日，CBS宣布續簽此劇三年延至2013-2014季度。在2014年3月，該劇再次獲得為期三季度的續約，使得該劇延至2016-17季度，達到10季。2017年3月，製作方宣布將續約兩季度，到2018-19季度，至12季。2018年8月，官方宣布第12季將為該劇的最後一季。

    """
]

In [22]:
from typing import List
import numpy as np

def get_answer(query: str, source: List[str]):
    most_related_sentence = None
    max_similarity = 0

    for sentence in source:
        sim = calculate_cosine_similarity(
        embedding_model.encode(query),
        embedding_model.encode(sentence)
        )

        if sim > max_similarity:
            most_related_sentence = sentence
            max_similarity = sim

    return most_related_sentence

def calculate_cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [ ]:
user_query = input(">>>")
conversation = Conversation()

while user_query.lower() != "bye":
    print(f"user: {user_query}")
    # 尋找最相似的文件
    answer = get_answer(user_query, qa_data)
    llm_input = f"""請你基於以下資訊回答使用者的問題
    {answer}
    ===
    問題：{user_query}
    """
    conversation.add_user_input(llm_input)
    # 將 conversation.messages 丟給 chatbot
    chatbot_result = chatbot(conversation.messages)[0]['generated_text']
    print(f"AI: {chatbot_result}")
    conversation.append_response(chatbot_result)

    user_query = input(">>>")


>>>Sheldon是誰
user: Sheldon是誰
AI: Sheldon 是《宅男行不行》（The Big Bang Theory）劇中的角色，由美國演員 Jim Parsons 飾演。他是一位天才物理學家，也是一位非常固執且社交能力不足的理論物理學家。他與好友 Leonard Hofstadter 共同住在 Pasadena 的 Apartment 41，兩人在劇中經常一起解決生活和工作上的問題。Sheldon 的角色在劇集中非常受歡迎，Jim Parsons 也因此角色獲得過多次艾美獎。
>>>Emily呢
user: Emily呢
AI: 問題：Emily呢？

在提供的資訊中，並沒有提到一名叫做 Emily 的角色或演員。可能使用者在尋找有關《宅男行不行》中的角色或角色演員，而提供的資訊中並沒有提到 Emily 這個角色或人物。
>>>Amy?
user: Amy?
AI: 問題：Amy?

Amy Farrah Fowler 是《宅男行不行》（The Big Bang Theory）劇中的角色，由演員 Melissa Rauch 飾演。她原本是由一個線上約會網站推薦給她朋友 Sheldon Cooper 的完美伴侶。Amy 來自 Glendale, California，擁有神經生物學博士學位。在劇中，Amy 最初與 Sheldon 共享社交困境，但在認識 Penny 和 Bernadette 後，她開始對社交和浪漫互動表示出更多興趣。Amy 和 Sheldon 的關係逐步進展，到最後他們承認彼此是朋友，然後變成男女朋友，最後更結成夫妻。
>>>Emily Sweeney?
user: Emily Sweeney?
AI: 問題：Emily Sweeney?

Emily Sweeney 是《宅男行不行》（The Big Bang Theory）劇中的角色，由演員 Laura Spencer 飾演。她曾是一名在 Huntington Hospital 工作的皮膚科醫生，來自哈佛，在劇中她喜歡拿刀切東西的感覺。Emily 最初與 Raj 因為線上約會平台認識，Raj 希望透過 Amy 作為他的「翼手」來追求 Emily。她與 Raj 的關係成為獨家，但最後因 Raj 對 Claire 情有獨鍾而分手。
